In [ ]:
"""
This file is part of lc-power-match-baluns.
Copyright © 2023 Technical University of Denmark (developed by Rasmus Jepsen)

This library is free software; you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public
License as published by the Free Software Foundation; either
version 2.1 of the License, or (at your option) any later version.

This library is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with this library; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301  USA
"""

In [ ]:
"""
This notebook uses Tippet's technique [1] to combine pairs of two-port scattering parameters measured with a VNA into three-port scattering parameters.
The code in this notebook was inspired by the process described in https://scikit-rf.readthedocs.io/en/latest/examples/metrology/Measuring%20a%20Mutiport%20Device%20with%20a%202-Port%20Network%20Analyzer.html
"""

In [ ]:
# import modules
import skrf as rf
%matplotlib inline
from pylab import *
import numpy as np
rf.stylely()

In [ ]:
# input for the name of the network to combine scattering parameters for
network_name = 'lattice'

In [ ]:
# read the two-port scattering parameters from Touchstone files
network12 = rf.Network(f'raw_data/{network_name} ports 1 2.s2p', 'network12', s_def='traveling')
network23 = rf.Network(f'raw_data/{network_name} ports 2 3.s2p', 'network23', s_def='traveling')
network13 = rf.Network(f'raw_data/{network_name} ports 1 3.s2p', 'network13', s_def='traveling')
frequencies = network12.frequency

In [ ]:
# populate the three-port scattering parameter matrix
full_s = np.zeros(shape=(len(frequencies), 3, 3), dtype=np.complex_)
full_s[:, 0, 0] = network12.s[:, 0, 0]
full_s[:, 0, 1] = network12.s[:, 0, 1]
full_s[:, 0, 2] = network13.s[:, 0, 1]
full_s[:, 1, 0] = network12.s[:, 1, 0]
full_s[:, 1, 1] = network12.s[:, 1, 1]
full_s[:, 1, 2] = network23.s[:, 0, 1]
full_s[:, 2, 0] = network13.s[:, 1, 0]
full_s[:, 2, 1] = network23.s[:, 1, 0]
full_s[:, 2, 2] = network23.s[:, 1, 1]
full_network = rf.Network(frequency=frequencies, s=full_s, z0=[50, 50, 50], s_def='traveling')

In [ ]:
# write the three-port scattering parameters into a Touchstone file
full_network.write_touchstone(f'{network_name}.s3p', dir=f'processed_data')

In [ ]:
"""References:
[11] J. Tippet and R. Speciale, "A rigorous technique for measuring the scattering matrix of a multiport device with a 2-port network analyzer," IEEE Transactions on Microwave Theory and Techniques, vol. 30, no. 5, pp. 661–666, 1982.
"""